In [1]:
import numpy as np
import pandas as pd
from math import *
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
from random import *
%matplotlib inline 

In [2]:
def sub_lists(l):
    base = []  
    lists = [base]
    for i in range(len(l)):
        orig = lists[:]
        new = l[i]
        for j in range(len(lists)):
            lists[j] = lists[j] + [new]
        lists = orig + lists
          
    return lists

In [36]:
########################################################################################################
# Creating a Cooperative game framework of N 
# The created games will be used to compare between the uniform sampling approach and the structured one
########################################################################################################
def Coop_game(N):
    a = 1
    b = 20
    players = [i for i in range(N)]
    coalitions = sub_lists(players)
    K = []
    for c in coalitions:
        K.append([c,randint(a, b)])
    K[0][1] = 0
    m = 0   
    for c in K:  # Normalization of the coalition's values
        m += c[1]
    for c in K:
        c[1] = c[1]/m
    
    x = Shap_exact(players, K)
    targets = x[0]
    d = x[1]
        
    return K, targets, players

In [37]:
######################################################
# The exact Shapley values computation for each player
######################################################
def Shap_exact(players, K):
    N = len(players)
    targets = {}
    for c in players:
        I = []
        for k in K:
            if c not in k[0]:
                I.append(k)    # List of coalitions U such as player not in U 
        d = len(I)        
        #### Shapley value computation
        Sh = 0
        Luc = []
        for s in I:
            for j in K:
                if set(j[0]) == set(s[0]+[c]):
                    Luc.append(j)
        for i in range(len(Luc)):
            alpha = (factorial(N-len(I[i][0])-1)*factorial(len(I[i][0])))/factorial(N)
            Sh += alpha*(Luc[i][1]-I[i][1])
        targets[c] = Sh
    return targets, d

In [53]:
######################################################################################################
# Simulation of p Cooperative games and computing the Shapley values for each player, for each Game
# and for each random coalitions sample size (parameter r_param)
######################################################################################################
def simulation(p, N, r_param):
    U = []
    targets = {}
    for t in range(p):
        N = 10 
        K, targets[t], players= Coop_game(N)

        D = {}
        for c in players:
            D[c] = []
            I = []
            for k in K:
                if c not in k[0]:
                    I.append(k)    # List of coalitions such as player not in these ones 
            d = len(I)        
            for r in range(10,d,r_param):
                L = sample(I,r)
                #### Shapley value computation
                Sh = 0
                Luc = []
                for s in L:
                    for j in K:
                        if set(j[0]) == set(s[0]+[c]):
                            Luc.append(j)
                for i in range(len(Luc)):
                    alpha = (factorial(N-len(L[i][0])-1)*factorial(len(L[i][0])))/factorial(N)
                    Sh += alpha*(Luc[i][1]-L[i][1])
                D[c].append([r,Sh])
#         print(t)
        for c in D:
            U.append([c,D[c]])
    return U, d, targets

In [76]:
##############################################################################################
# Computing the AAPE (average average percentage error) for each random coalitions sample size
# and evaluating its behaviour
##############################################################################################
# U == simulation(p, N, r)
# targets == simulation(p, N, r_param)[2]
def AAPE_computation(U, targets, players, d, Nbr_games, r_param):
    ##
    S = {}
    for c in players:
        S[str(c)] = []
        for p in U:
            if p[0] == c:
                S[str(c)].append(p[1])
    ##
    
    AAPE = {}
    L = [i for i in range(10,d,r_param)]
    for c in L:
        W1 = {}
        for p1 in players:
            W1[p1] = []
            for p2 in range(Nbr_games):
                for k in S[str(p1)][p2]:
                    if k[0] == c:
                        W1[p1].append(k[1])
        m1 = 0
        for sim in range(Nbr_games):
            m2 = 0
            for t in players:
                m2 += abs(W1[t][sim]-targets[sim][t])/abs(targets[sim][t])
            m2 = m2/len(players)
            m1 += m2

        AAPE[str(c)] = m1/Nbr_games
    return AAPE